In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from sklearn.feature_selection import mutual_info_classif
import numpy as np
from sklearn.decomposition import PCA

# Load data with PANDAS
* TO DO: write the function load() using pandas.read_csv

In [3]:
def load(path):
    return pd.read_csv(path)

TrainXpath="EmberXTrain.csv"
TrainYpath="EmberYTrain.csv"
X=load(TrainXpath)
Y=load(TrainYpath)

# Pre-elaborate data with PANDAS
* For each independent variable analyze the distribution of the values
* Explore the class distribution in Y
* Remove the useless variables

In [4]:
def preElaborationData(X):\

    return X.describe()

description = preElaborationData(X)

In [5]:
def delete_zero_mean_columns(df):
    
    means = preElaborationData(df).loc['mean']
    
    zero_mean_columns = means[means == 0].index
    
    df = df.drop(columns=zero_mean_columns)
    
    return df

X = delete_zero_mean_columns(X)

# Feature evaluation with sklearn

In [6]:
def preBoxPlotAnalysisData(X, Y):
    
    output_directory = 'boxplot'

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    num_columns = len(X.columns)
    for i, column in enumerate(tqdm(X.columns, desc="Progress")):
        data = pd.DataFrame(X[column])
        data['Label'] = Y['Label']

        fig, ax = plt.subplots()
        data.boxplot(by='Label', ax=ax)
        ax.set_title(f'Boxplot for {column}')

        output_path = os.path.join(output_directory, f'boxplot_{column}.png')
        plt.savefig(output_path)
        plt.close()

#preBoxPlotAnalysisData(X, Y)

In [7]:
def mutualInfoRank(X,Y):
    
    seed=42
    np.random.seed(seed)
    print("Computing mutual info ranking...")
    independentList=list(X.columns.values)
    res = dict(zip(independentList,mutual_info_classif(X, np.ravel(Y),discrete_features=False, random_state=seed)))
    sorted_x = sorted(res.items(), key=lambda kv: kv[1], reverse=True)
    print("Computing mutual info ranking...completed")
    return sorted_x

rank=mutualInfoRank(X,Y)
print(rank)

Computing mutual info ranking...
Computing mutual info ranking...completed
[('directories_2355', 0.34468200595838727), ('head_627', 0.3351517763161995), ('directories_2356', 0.32913839267328804), ('section_837', 0.3138364258706954), ('gen_617', 0.31162093000689106), ('section_787', 0.2781273648951521), ('byte_511', 0.27104537979028587), ('byte_509', 0.2702485845587479), ('byte_499', 0.27017901474027317), ('byte_510', 0.26964703490355557), ('byte_512', 0.26843151886172256), ('byte_503', 0.2683192359687421), ('head_686', 0.2680030306027956), ('byte_502', 0.2677204558277211), ('byte_508', 0.2676981194156387), ('byte_507', 0.2670991610051885), ('byte_500', 0.2670256127775097), ('byte_501', 0.26688244537579675), ('byte_504', 0.2666026041661853), ('hist_140', 0.2663814636397459), ('byte_497', 0.26563907518056395), ('byte_506', 0.26350830374861434), ('hist_138', 0.26342256264302244), ('byte_498', 0.2624291254547646), ('hist_232', 0.2618114229365578), ('byte_505', 0.261676750186558), ('hist_23

# Feature selection
* Write the function topFeatureSelect that returns the top features ranked according to MI with MI>=threshold
* To test the function, build XSelected that is the projection of X on the features with MI>=0.1

In [8]:
def topFeatureSelect(rank, threshold):

    selected_features = []

    for feature in rank:
        mi = feature[1]
        if mi>=threshold:
            selected_features.append(feature[0])
    
    return selected_features

# selecting top features
selectedfeatures=topFeatureSelect(rank,0.1)
# create a dataset with the selected features
XSelected=X.loc[:, selectedfeatures]

# PCA
* TO DO: write Python function(s) to learn the principal components of a trainingdata set, in order to create a new data frame projecting the training dataset onboth the top-N principal components and the label
* https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html check fit() and transform()
1) Note that the principal component model learned on the training set should be also saved to be applied to a possible testing set, when this will be available
2) The PCA is computed on the independent features only

* Write  the  function  pca  that  takes  as  input  a  Dataframe  and  return  the  PCA 
(computed with fit on the Dataframe), the list  of names «pc1», «pc2»,... and the 
list of the explained variance of the PCA object (recorded in 
explained_variance_ratio_)
* Write the function applyPCA that takes as input a Dataframe, a PCA and a list of 
PC names, transform the Dataframe using PCA and create a data frame collecting 
the principal components obtained by using transform of PCA on the Dataframe. 
The  names  of  the  columns  of  the  output    Dataframe  are    the  same  reported  in 
the list of PC names


In [13]:
def pca(df):
    pca = PCA().fit(df)
    pcalist = [f'PC{i+1}' for i in range(len(pca.components_))]
    explained_variance = pca.explained_variance_ratio_
    return pca, pcalist, explained_variance

def applyPCA(df, pca_model, pc_names):
    pcs = pca_model.transform(df)
    pc_df = pd.DataFrame(data=pcs, columns=pc_names)
    return pc_df

pca_model, pcalist, explained_variance = pca(X)
XPCA = applyPCA(X, pca_model, pcalist)


[9.17334959e-01 5.07081373e-02 1.33576822e-02 ... 9.07360335e-33
 1.78412937e-36 5.58902635e-37]
['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26', 'PC27', 'PC28', 'PC29', 'PC30', 'PC31', 'PC32', 'PC33', 'PC34', 'PC35', 'PC36', 'PC37', 'PC38', 'PC39', 'PC40', 'PC41', 'PC42', 'PC43', 'PC44', 'PC45', 'PC46', 'PC47', 'PC48', 'PC49', 'PC50', 'PC51', 'PC52', 'PC53', 'PC54', 'PC55', 'PC56', 'PC57', 'PC58', 'PC59', 'PC60', 'PC61', 'PC62', 'PC63', 'PC64', 'PC65', 'PC66', 'PC67', 'PC68', 'PC69', 'PC70', 'PC71', 'PC72', 'PC73', 'PC74', 'PC75', 'PC76', 'PC77', 'PC78', 'PC79', 'PC80', 'PC81', 'PC82', 'PC83', 'PC84', 'PC85', 'PC86', 'PC87', 'PC88', 'PC89', 'PC90', 'PC91', 'PC92', 'PC93', 'PC94', 'PC95', 'PC96', 'PC97', 'PC98', 'PC99', 'PC100', 'PC101', 'PC102', 'PC103', 'PC104', 'PC105', 'PC106', 'PC107', 'PC108', 'PC109', 'PC110', 'PC111', 'PC112', 'P

* Write the function NumberOfTopPCSelect that returns the principal
components achieving the sum of variance greater than a threshold
* Test the function by computin XPCASelected

In [14]:
def NumberOfTopPCSelect(explained_variance, threshold):
    total_variance = 0
    num_selected = 0
    
    for variance in explained_variance:
        total_variance += variance
        num_selected += 1
        if total_variance >= threshold:
            break
    
    return num_selected

n=NumberOfTopPCSelect(explained_variance,0.99)
print(n)
# create a dataset with the selected PCs
XPCASelected=XPCA.iloc[:,1:(n+1)]
print(XPCASelected.shape)


5
(12000, 5)
